In [1]:
#Load packages and prepare the header statement
from bs4 import BeautifulSoup
import re
import requests
import time
import random
import csv
import datetime 
import numpy as np
import os, sys
import os.path
import urllib.request
from stem import Signal
from stem.control import Controller
import pandas as pd
from random import randint
from time import sleep


headers = {
    'Connection': 'keep-alive',
    'Access-Control-Request-Headers': 'content-type',
    'Accept': '*/*',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'
}


In [3]:
#Start tor.exe and Privoxy.exe to enable IP Rotation

class ConnectionManager:
    def __init__(self):
        self.new_ip = "0.0.0.0"
        self.old_ip = "0.0.0.0"
        self.new_identity()

    @classmethod
    def _get_connection(self):
        """
        TOR new connection
        """
        with Controller.from_port(port=9051) as controller:
            controller.authenticate(password="1234")
            controller.signal(Signal.NEWNYM)
            controller.close()

    @classmethod
    def _set_url_proxy(self):
        """
        Request to URL through local proxy
        """
        proxy_support = urllib.request.ProxyHandler({"http": "127.0.0.1:8118"})
        opener = urllib.request.build_opener(proxy_support)
        urllib.request.install_opener(opener)

    @classmethod
    def request(self, url):
        """
        TOR communication through local proxy
        :param url: web page to parser
        :return: request
        """
        try:
            self._set_url_proxy()
            request = urllib.request.Request(url, None, {
                'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) "
                              "AppleWebKit/535.11 (KHTML, like Gecko) "
                              "Ubuntu/10.10 Chromium/17.0.963.65 "
                              "Chrome/17.0.963.65 Safari/535.11"})
            request = urllib.request.urlopen(request)
            return request
        except urllib.request.HTTPError:
            return e.message

    def new_identity(self):
        """
        new connection with new IP
        """
        # First Connection
        if self.new_ip == "0.0.0.0":
            self._get_connection()
            self.new_ip = self.request("http://icanhazip.com/").read()
        else:
            self.old_ip = self.new_ip
            self._get_connection()
            self.new_ip = self.request("http://icanhazip.com/").read()

        seg = 0

        # If we get the same ip, we'll wait 5 seconds to request a new IP
        while self.old_ip == self.new_ip:
            time.sleep(5)
            seg += 5
            print ("Waiting to obtain new IP: %s Seconds" % seg                                   ,  end="\r")
            self.new_ip = self.request("http://icanhazip.com/").read()
        
cm = ConnectionManager()

In [4]:
#Main Code 

#Number of pages with multiple cars 

dic = {}
t = 0

cm.new_identity()

with open('AutoTraderLinks.csv', 'a', newline='') as csvfile:
    AutroTraderCSV = csv.writer(csvfile)
    averagetime = []

    #Build in another for loop to incorporate the brand of the car. We can only scrape till 100 pages.
    #Use the page indication to find the maximum number of pages.
    #Maybe only get information from the top 10 brands up to a 1000 observations, sorted on the newests cars to get
    #the most recent information
    
    #For example, Land Rover, Maza, Audi, Mercedes, Mini, Nissan, Citroen, Peugeot, Renault, Skoda, Ford,
    #Hyundai, Toyota, Vauxhall, Jaguar, Volkswagen, Volvo and Kia. 
    
    #Set number to scrape to the maximum number of pages or equal to 100 if the number is larger. 
    numbertoscrape = 6000
    
    for i in np.arange(1, numbertoscrape, 1):
        linklist
            
        response = requests.get('https://www.autotrader.co.uk/car-search?sort=sponsored&radius=1500&postcode=ec1a1yx&onesearchad=Nearly%20New&page='+str(i), headers=headers).text
        AutoTrader = BeautifulSoup(response, 'html.parser')
        Cars = AutoTrader.find_all('a', {'class': 'listing-fpa-link'})
        linklist = []
        for tag in Cars:
            link = [tag.get('href')]
            for tag in link:
                 linklist = linklist + link
        linklist = np.unique(linklist[5:43]) 
        
        if i % 2 == 0:
                cm.new_identity()
        
        for x in np.arange(1,10):
            start = datetime.datetime.now()
            sleep(randint(1,6))
            response2 = requests.get('https://www.autotrader.co.uk'+str(linklist[x]), headers=headers).text
            CarSpecs = BeautifulSoup(response2, 'html.parser')

            try:
                dic['CarName'] = CarSpecs.find("span", {"class": "pricetitle__advertTitle"}).text.split(',')[0].strip()
            except:
                dic['CarName'] = ""   
            try:
                dic['CarPrice'] = CarSpecs.find("section", {"class": "priceTitle__price gui-advert-price"}).text.strip()
            except:
                dic['CarPrice'] = ""    
            try:
                dic['WebRating'] = CarSpecs.find("div", {"class": "starRating__number"}).text.strip()
            except:
                dic['WebRating'] = "" 
                        
            TableCar = CarSpecs.find_all("ul", {"class": "keyFacts__list"})
            for tag in TableCar:
                spec = tag.text
                spec = spec.splitlines() 

            try:
                dic['Year'] = spec[1] 
            except:
                dic['Year'] = "" 
            try:
                dic['CarType'] = spec[2] 
            except:
                dic['CarType'] = "" 
            try:
                dic['Miles'] = spec[3] 
            except:
                dic['Miles'] = "" 
            try:
                dic['Transmission'] = spec[4] 
            except:
                dic['Transmission'] = "" 
            try:
                dic['EngineDiscplacement'] = spec[5]
            except:
                dic['EngineDiscplacement'] = "" 
            try:
                dic['FuelType'] = spec[6]
            except:
                dic['FuelType'] = "" 
                
                
            RunningCosts = CarSpecs.find_all("div", {"class": "fpaSpecifications__description"})
            RC = []
            for tag in RunningCosts:
                RC = RC + [tag.text]
            
            try:
                dic['UrbanMPG'] = RC[0]
            except:
                dic['UrbanMPG'] = ""
            try:
                dic['Extra Urban mpg'] = RC[1] 
            except:
                dic['Extra Urban mpg'] = ""
            try:    
                dic['Average mpg'] = RC[2] 
            except:
                dic['Average mpg'] = ""
            try:
                dic['CO2 emissions'] = RC[3] 
            except:
                dic['CO2 emissions'] = ""
            try:
                dic['Annual Tax'] = RC[4] 
            except:
                dic['Annual Tax'] = ""
            try:
                dic['Engine power'] = RC[5] 
            except:
                dic['Engine power'] = ""
            try:
                dic['Engine size'] = RC[6]
            except:
                dic['Engine size'] = ""
            try:
                dic['Brochure Engine size'] = RC[7] 
            except:
                dic['Brochure Engine size'] = ""
            try:
                dic['Acceleration (0-60mph)'] = RC[8] 
            except:
                dic['Acceleration (0-60mph)'] = ""
            try:
                dic['Top speed'] = RC[9] 
            except:
                dic['Top speed'] = ""
            try:
                dic['Drivetrain'] = RC[10] 
            except:
                dic['Drivetrain'] = ""
            try:
                dic['Nr Doors'] = RC[11] 
            except:
                dic['Nr Doors'] = ""
            try:
                dic['Nr Seats'] = RC[12] 
            except:
                dic['Nr Seats'] = ""   
                
            Options = CarSpecs.find_all("div", {"class": "fpaSpecifications__term"})
            Op = []
            for tag in Options:
                Op = Op + [tag.text]

            dic['Options'] = Op[13:]
    
            if t == 0:
                AutroTraderCSV.writerow(dic.keys())
                AutroTraderCSV.writerow(dic.values()) 
                t +=1
            else:
                AutroTraderCSV.writerow(dic.values())
            
            end = datetime.datetime.now()    
            averagetime += [(end - start).total_seconds()]
            meantime = np.mean(averagetime)
            timeleft = str(datetime.timedelta(seconds=((numbertoscrape*9-((i-1)*9+x)) * meantime)))

            print(str(round(((i-1)*9+x)/(numbertoscrape*9),2)*100)+'% Complete, on number :' +str((i-1)*9+x)+ ' , Time remaining: ' + str(timeleft)," Scraping on IP adress "+ str((cm.new_ip).strip()),end="\r")               

IndexError: index 1 is out of bounds for axis 0 with size 0